**데이터 로드 및 전처리**

In [ ]:
# 데이터 로드 및 전처리 코드
import pandas as pd
from google.colab import drive
from sklearn.preprocessing import LabelEncoder

# Google Drive 마운트
drive.mount('/content/drive')

# 데이터셋 로드 (euc-kr 인코딩 사용)
subway_data = pd.read_csv('/content/drive/MyDrive/2022subwaydata.csv', encoding='euc-kr')
weather_data = pd.read_csv('/content/drive/MyDrive/2022weather.csv', encoding='euc-kr')

# 날짜 열을 datetime 형식으로 변환
subway_data['date'] = pd.to_datetime(subway_data['수송일자'])
weather_data['forecast_date'] = pd.to_datetime(weather_data['예보일자'])

# 두 데이터셋을 날짜를 기준으로 병합
merged_data = pd.merge(subway_data, weather_data, left_on='date', right_on='forecast_date', how='inner')

# 결측치가 너무 많은 열 삭제
merged_data = merged_data.drop(columns=['24시이후'])

# 주말/평일을 수치형으로 변환 (평일=0, 주말=1)
merged_data['weekday_weekend'] = merged_data['평일주말'].map({'평일': 0, '주말': 1})

# 강수/비강수를 수치형으로 변환 (비강수=0, 강수=1)
merged_data['weather_numeric'] = merged_data['예보'].map({'비강수': 0, '강수': 1})

# '호선'과 '역명'을 수치형으로 변환
le_line = LabelEncoder()
le_station = LabelEncoder()

merged_data['호선_encoded'] = le_line.fit_transform(merged_data['호선'])
merged_data['역명_encoded'] = le_station.fit_transform(merged_data['역명'])

# 데이터 전처리 완료
print(merged_data.head())


Mounted at /content/drive
         수송일자  호선    역명 승하차구분  06시이전  06-07시간대  07-08시간대  08-09시간대  09-10시간대  \
0  2022-01-01   1   서울역    하차    113       560       617       910      1232   
1  2022-01-01   1    시청    하차     31       195       224       380       283   
2  2022-01-01   1    종각    하차     33       136       199       334       489   
3  2022-01-01   1  종로3가    하차     43       139       124       272       414   
4  2022-01-01   1  종로5가    하차     14        80       137       239       374   

   10-11시간대  ...  23-24시간대       date        예보일자  평일주말   예보  forecast_date  \
0      1260  ...       131 2022-01-01  2022-01-01    주말  비강수     2022-01-01   
1       309  ...        24 2022-01-01  2022-01-01    주말  비강수     2022-01-01   
2       604  ...        40 2022-01-01  2022-01-01    주말  비강수     2022-01-01   
3       658  ...        47 2022-01-01  2022-01-01    주말  비강수     2022-01-01   
4       482  ...        41 2022-01-01  2022-01-01    주말  비강수     2022-01-01   

   weekday_weekend

**3가지 모델 학습 코드**

In [ ]:
# 필요한 라이브러리 설치 및 임포트
!pip install xgboost
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor  # XGBoost 추가
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import joblib

# 예측에 사용할 특성 선택
features = ['호선_encoded', '역명_encoded', 'weekday_weekend', 'weather_numeric']
time_columns = ['06-07시간대', '07-08시간대', '08-09시간대', '09-10시간대',
                '10-11시간대', '11-12시간대', '12-13시간대', '13-14시간대',
                '14-15시간대', '15-16시간대', '16-17시간대', '17-18시간대',
                '18-19시간대', '19-20시간대', '20-21시간대', '21-22시간대',
                '22-23시간대', '23-24시간대']

# 특성 데이터 X와 타겟 데이터 y 설정
X = merged_data[features]
y = merged_data[time_columns]

# 데이터 스케일링
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# 학습/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

### RandomForest 모델 학습
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

### 선형 회귀 모델 학습
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

### XGBoost 모델 학습
xgb_model = XGBRegressor(random_state=42)
xgb_model.fit(X_train, y_train)

# 학습 완료, 모델 저장
joblib.dump(rf_model, '/content/drive/MyDrive/random_forest_model.pkl')
joblib.dump(lr_model, '/content/drive/MyDrive/linear_regression_model.pkl')
joblib.dump(xgb_model, '/content/drive/MyDrive/xgboost_model.pkl')
joblib.dump(scaler, '/content/drive/MyDrive/scaler.pkl')

print("모델 학습 및 저장 완료")

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 모델 학습 후 평가 함수 정의
def evaluate_model(model, X_test, y_test):
    # 예측 수행
    y_pred = model.predict(X_test)

    # 평가 지표 계산
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"MSE (Mean Squared Error): {mse}")
    print(f"MAE (Mean Absolute Error): {mae}")
    print(f"R² Score: {r2}")

# RandomForest 모델 학습 후 평가
print("RandomForest 모델 평가 결과:")
evaluate_model(rf_model, X_test, y_test)

# XGBoost 모델 학습 후 평가
print("\nXGBoost 모델 평가 결과:")
evaluate_model(xgb_model, X_test, y_test)

# 선형 회귀 모델 학습 후 평가
print("\nLinear Regression 모델 평가 결과:")
evaluate_model(lr_model, X_test, y_test)



모델 학습 및 저장 완료
RandomForest 모델 평가 결과:
MSE (Mean Squared Error): 49206.54445202649
MAE (Mean Absolute Error): 105.20189974091514
R² Score: 0.9196633355521993

XGBoost 모델 평가 결과:
MSE (Mean Squared Error): 50057.03492209429
MAE (Mean Absolute Error): 108.55569492194131
R² Score: 0.918847918510437

Linear Regression 모델 평가 결과:
MSE (Mean Squared Error): 856919.648724132
MAE (Mean Absolute Error): 506.679719385899
R² Score: 0.14631282086738123


**하이퍼파라미터 튜닝 및 최적화**

In [ ]:
# 하이퍼파라미터 튜닝 및 최적 모델 저장 코드
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor  # XGBoost 추가
from sklearn.ensemble import RandomForestRegressor
import joblib

# 하이퍼파라미터 그리드 설정 (XGBoost와 RandomForest만 사용)
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10]
}

param_grid_xgb = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 10],
    'learning_rate': [0.01, 0.1, 0.2]
}

# RandomForest 모델 초기화 및 하이퍼파라미터 튜닝
grid_search_rf = GridSearchCV(estimator=RandomForestRegressor(random_state=42), param_grid=param_grid_rf, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_rf.fit(X_train, y_train)

# XGBoost 모델 초기화 및 하이퍼파라미터 튜닝
grid_search_xgb = GridSearchCV(estimator=XGBRegressor(random_state=42), param_grid=param_grid_xgb, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_xgb.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print(f"RandomForest 최적의 하이퍼파라미터: {grid_search_rf.best_params_}")
print(f"XGBoost 최적의 하이퍼파라미터: {grid_search_xgb.best_params_}")

# 최적 모델로 예측
best_rf_model = grid_search_rf.best_estimator_
best_xgb_model = grid_search_xgb.best_estimator_

y_pred_rf = best_rf_model.predict(X_test)
y_pred_xgb = best_xgb_model.predict(X_test)

# 성능 평가 (MSE 계산)
mse_rf = mean_squared_error(y_test, y_pred_rf)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)

print(f'RandomForest 최적화된 모델의 MSE: {mse_rf}')
print(f'XGBoost 최적화된 모델의 MSE: {mse_xgb}')

# 최적 모델 결정 및 저장
if mse_rf < mse_xgb:
    best_model = best_rf_model
    print("RandomForest 모델이 최적화된 모델로 선택되었습니다.")
    joblib.dump(best_model, '/content/drive/MyDrive/optimized_random_forest_model.pkl')
else:
    best_model = best_xgb_model
    print("XGBoost 모델이 최적화된 모델로 선택되었습니다.")
    joblib.dump(best_model, '/content/drive/MyDrive/optimized_xgboost_model.pkl')

# 스케일러 저장
joblib.dump(scaler, '/content/drive/MyDrive/scaler.pkl')

print("최적화된 모델과 스케일러 저장 완료")


RandomForest 최적의 하이퍼파라미터: {'max_depth': 20, 'min_samples_split': 10, 'n_estimators': 300}
XGBoost 최적의 하이퍼파라미터: {'learning_rate': 0.2, 'max_depth': 10, 'n_estimators': 100}
RandomForest 최적화된 모델의 MSE: 49199.02163252827
XGBoost 최적화된 모델의 MSE: 49171.42801775981
XGBoost 모델이 최적화된 모델로 선택되었습니다.
최적화된 모델과 스케일러 저장 완료


**예측코드**

In [ ]:
# 예측 수행 코드
import numpy as np
import pandas as pd
import joblib

# 최적화된 모델과 스케일러 로드
best_model = joblib.load('/content/drive/MyDrive/optimized_random_forest_model.pkl')  # 최적화된 모델 (RandomForest or XGBoost)
scaler = joblib.load('/content/drive/MyDrive/scaler.pkl')

# LabelEncoder를 다시 학습시키기 위해 필요한 데이터 로드 (subway_data)
subway_data = pd.read_csv('/content/drive/MyDrive/2022subwaydata.csv', encoding='euc-kr')

# '호선'과 '역명'을 학습시키기 위한 LabelEncoder 재학습
from sklearn.preprocessing import LabelEncoder
le_line = LabelEncoder()
le_station = LabelEncoder()
le_line.fit(subway_data['호선'])
le_station.fit(subway_data['역명'])

# 호선 및 역명을 수치형으로 변환하는 함수
def encode_line_station(line_name, station_name):
    line_encoded = le_line.transform([line_name])[0]
    station_encoded = le_station.transform([station_name])[0]
    return line_encoded, station_encoded

# 예측 함수 정의
def predict_future_passengers(model, date, is_weekend, is_rain, line_name, station_name):
    time_columns = ['06-07시간대', '07-08시간대', '08-09시간대', '09-10시간대',
                    '10-11시간대', '11-12시간대', '12-13시간대', '13-14시간대',
                    '14-15시간대', '15-16시간대', '16-17시간대', '17-18시간대',
                    '18-19시간대', '19-20시간대', '20-21시간대', '21-22시간대',
                    '22-23시간대', '23-24시간대']

    # 주말/평일 여부는 0: 평일, 1: 주말
    weekday_weekend = 1 if is_weekend else 0

    # 비 여부는 0: 비 안 옴, 1: 비 옴
    rain = 1 if is_rain else 0

    # 호선과 역명 인코딩
    line_encoded, station_encoded = encode_line_station(line_name, station_name)

    # 예측을 위한 입력 데이터 생성
    prediction_data = np.zeros((1, 4))  # 호선, 역명, 주말 여부, 비 여부에 대한 특성만 사용

    # 특성 데이터에 대한 설정 (호선, 역명, 주말 여부, 날씨)
    prediction_data[0, 0] = line_encoded  # 호선 설정
    prediction_data[0, 1] = station_encoded  # 역명 설정
    prediction_data[0, 2] = weekday_weekend  # 주말 여부 설정
    prediction_data[0, 3] = rain  # 비 여부 설정

    # 스케일링 적용 (사전에 학습된 스케일러 사용)
    scaled_data = scaler.transform(prediction_data)

    # 모델을 이용해 시간대별 승객 수 예측
    predicted_passengers = model.predict(scaled_data)

    # 결과를 데이터프레임으로 변환
    result_df = pd.DataFrame({'Time Slot': time_columns, 'Predicted Passengers': predicted_passengers[0]})

    return result_df

# 예측 예시: 2024년 10월 4일 2호선 건대입구역
date = '2024-10-04'
is_weekend = 1  # 주말
is_rain = 0  # 비가 오지 않음
line_name = '2'  # 2호선
station_name = '건대입구'  # 건대입구역

# 최적화된 모델을 사용하여 예측 수행
predicted_passengers = predict_future_passengers(best_model, date, is_weekend, is_rain, line_name, station_name)

# 예측 결과 출력
print("최적화된 모델 예측 결과:")
print(predicted_passengers)


최적화된 모델 예측 결과:
   Time Slot  Predicted Passengers
0   06-07시간대            390.679057
1   07-08시간대            341.374337
2   08-09시간대            568.956080
3   09-10시간대            987.604324
4   10-11시간대           1180.807309
5   11-12시간대           1699.166621
6   12-13시간대           2170.261004
7   13-14시간대           2478.140882
8   14-15시간대           2461.668576
9   15-16시간대           2637.844471
10  16-17시간대           3109.306837
11  17-18시간대           3523.727088
12  18-19시간대           3289.474216
13  19-20시간대           2346.830409
14  20-21시간대           1843.465585
15  21-22시간대           1801.843744
16  22-23시간대           1454.021256
17  23-24시간대            943.099129


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


**호선별 시간대별 최대 승객 수 예측 코드**

In [ ]:
import warnings
import pandas as pd

# 불필요한 경고 메시지 숨기기
warnings.filterwarnings("ignore", category=UserWarning, module='sklearn')

# 소수점 표시 없이 출력 설정
pd.options.display.float_format = '{:.0f}'.format

# 각 호선별로 시간대별 최대 승객 수를 가진 역의 예측 함수
def predict_max_passengers_for_line(model, date, is_weekend, is_rain, line_name):
    # 시간대 정보 정의
    time_columns = ['06-07시간대', '07-08시간대', '08-09시간대', '09-10시간대',
                    '10-11시간대', '11-12시간대', '12-13시간대', '13-14시간대',
                    '14-15시간대', '15-16시간대', '16-17시간대', '17-18시간대',
                    '18-19시간대', '19-20시간대', '20-21시간대', '21-22시간대',
                    '22-23시간대', '23-24시간대']

    # 해당 호선에 속한 모든 역명 가져오기
    line_stations = subway_data[subway_data['호선'] == int(line_name)]['역명'].unique()

    if len(line_stations) == 0:
        print(f"{line_name}호선에 해당하는 역이 없습니다.")
        return pd.DataFrame({'Time Slot': time_columns, 'Max Station Predicted Passengers': [np.nan] * len(time_columns)})

    max_station = None
    max_passengers = None

    # 각 역에 대해 시간대별 승객 수 예측 후, 가장 많은 승객 수를 가진 역 찾기
    for station in line_stations:
        try:
            predicted_passengers = predict_future_passengers(model, date, is_weekend, is_rain, line_name, station)
            total_passengers = predicted_passengers['Predicted Passengers'].sum()  # 총 승객 수 계산

            if max_passengers is None or total_passengers > max_passengers:
                max_passengers = total_passengers
                max_station = station  # 승객 수가 가장 많은 역 갱신
        except Exception as e:
            continue

    if max_station is None:
        print(f"{line_name}호선에서 예측된 최대 승객 수를 가진 역이 없습니다.")
        return pd.DataFrame({'Time Slot': time_columns, 'Max Station Predicted Passengers': [np.nan] * len(time_columns)})

    # 가장 많은 승객 수를 가진 역의 예측 결과 출력
    max_station_predictions = predict_future_passengers(model, date, is_weekend, is_rain, line_name, max_station)
    max_station_predictions.columns = ['Time Slot', 'Max Station Predicted Passengers']

    print(f"{line_name}호선에서 가장 많은 승객 수를 가진 역은 '{max_station}'입니다.")
    return max_station_predictions

# 각 호선에 대해 시간대별 최대 승객 수를 가진 역의 예측 결과 출력
for line in range(1, 9):  # 1호선부터 8호선까지
    max_passengers_df = predict_max_passengers_for_line(best_model, date, is_weekend=1, is_rain=0, line_name=str(line))

    print(f"\n{line}호선 시간대별 최대 승객 수를 가진 역의 예측:")
    print(max_passengers_df)


1호선에서 가장 많은 승객 수를 가진 역은 '서울역'입니다.

1호선 시간대별 최대 승객 수를 가진 역의 예측:
   Time Slot  Max Station Predicted Passengers
0   06-07시간대                              1079
1   07-08시간대                              1227
2   08-09시간대                              1498
3   09-10시간대                              2170
4   10-11시간대                              2048
5   11-12시간대                              2010
6   12-13시간대                              2528
7   13-14시간대                              2459
8   14-15시간대                              2340
9   15-16시간대                              2407
10  16-17시간대                              2386
11  17-18시간대                              2500
12  18-19시간대                              2109
13  19-20시간대                              1830
14  20-21시간대                              1503
15  21-22시간대                              1319
16  22-23시간대                               857
17  23-24시간대                               435
2호선에서 가장 많은 승객 수를 가진 역은 '홍대입구'입니다.

2호선 시간대별

In [ ]:
import warnings
import pandas as pd
import numpy as np

# 불필요한 경고 메시지 숨기기
warnings.filterwarnings("ignore", category=UserWarning, module='sklearn')

# 소수점 표시 없이 출력 설정
pd.options.display.float_format = '{:.0f}'.format

# 출퇴근 시간대 설정
peak_time_columns = ['07-08시간대', '08-09시간대', '09-10시간대', '17-18시간대', '18-19시간대', '19-20시간대']

# 각 호선별로 출퇴근 시간대에 승객 수가 가장 많은 역의 예측 함수
def predict_peak_passengers_for_line(model, date, is_weekend, is_rain, line_name):
    # 해당 호선에 속한 모든 역명 가져오기
    line_stations = subway_data[subway_data['호선'] == int(line_name)]['역명'].unique()

    if len(line_stations) == 0:
        print(f"{line_name}호선에 해당하는 역이 없습니다.")
        return pd.DataFrame({'Peak Time Slot': peak_time_columns, 'Max Station Predicted Passengers': [np.nan] * len(peak_time_columns)})

    max_station = None
    max_passengers = None

    # 각 역에 대해 출퇴근 시간대 승객 수 예측 후, 가장 많은 승객 수를 가진 역 찾기
    for station in line_stations:
        try:
            # 시간대별 승객 수 예측
            predicted_passengers = predict_future_passengers(model, date, is_weekend, is_rain, line_name, station)

            # 출퇴근 시간대만 추출
            peak_passengers = predicted_passengers[predicted_passengers['Time Slot'].isin(peak_time_columns)]

            # 출퇴근 시간대 총 승객 수 계산
            total_peak_passengers = peak_passengers['Predicted Passengers'].sum()

            # 가장 많은 승객 수를 가진 역 갱신
            if max_passengers is None or total_peak_passengers > max_passengers:
                max_passengers = total_peak_passengers
                max_station = station
        except Exception as e:
            print(f"{station} 예측 실패: {e}")
            continue

    if max_station is None:
        print(f"{line_name}호선에서 예측된 최대 승객 수를 가진 역이 없습니다.")
        return pd.DataFrame({'Peak Time Slot': peak_time_columns, 'Max Station Predicted Passengers': [np.nan] * len(peak_time_columns)})

    # 가장 많은 승객 수를 가진 역의 출퇴근 시간대 예측 결과 반환
    max_station_predictions = predict_future_passengers(model, date, is_weekend, is_rain, line_name, max_station)
    peak_max_station_predictions = max_station_predictions[max_station_predictions['Time Slot'].isin(peak_time_columns)]
    peak_max_station_predictions.columns = ['Peak Time Slot', 'Max Station Predicted Passengers']

    print(f"{line_name}호선에서 출퇴근 시간대 가장 많은 승객 수를 가진 역은 '{max_station}'입니다.")
    return peak_max_station_predictions

# 각 호선에 대해 출퇴근 시간대 최대 승객 수를 가진 역의 예측 결과 출력
for line in range(1, 9):  # 1호선부터 8호선까지
    print(f"\n{line}호선 출퇴근 시간대 최대 승객 수를 가진 역 예측 결과:")
    max_passengers_df = predict_peak_passengers_for_line(best_model, date, is_weekend=1, is_rain=0, line_name=str(line))
    print(max_passengers_df)



1호선 출퇴근 시간대 최대 승객 수를 가진 역 예측 결과:
1호선에서 출퇴근 시간대 가장 많은 승객 수를 가진 역은 '서울역'입니다.
   Peak Time Slot  Max Station Predicted Passengers
1        07-08시간대                              1227
2        08-09시간대                              1498
3        09-10시간대                              2170
11       17-18시간대                              2500
12       18-19시간대                              2109
13       19-20시간대                              1830

2호선 출퇴근 시간대 최대 승객 수를 가진 역 예측 결과:
2호선에서 출퇴근 시간대 가장 많은 승객 수를 가진 역은 '잠실(송파구청)'입니다.
   Peak Time Slot  Max Station Predicted Passengers
1        07-08시간대                              1036
2        08-09시간대                              1997
3        09-10시간대                              3664
11       17-18시간대                              4726
12       18-19시간대                              3847
13       19-20시간대                              2731

3호선 출퇴근 시간대 최대 승객 수를 가진 역 예측 결과:
3호선에서 출퇴근 시간대 가장 많은 승객 수를 가진 역은 '고속터미널'입니다.
   Peak Time Slot  Max Station Predic